# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
#Your code here; create the connection and cursor
conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

In [3]:
c.execute('''select * from products''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00


In [4]:
c.execute('''select * from orderdetails''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4


In [5]:
ls

CONTRIBUTING.md  data.sqlite  images/  index.ipynb  LICENSE.md  README.md


## Select the Average Number of Orders on a Product by Product Basis

In [6]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""select AVG(numOrder)
                    from (select productCode,
                                 count(distinct orderNumber) as numOrder
                                 from products
                                 join orderdetails
                                 using(productCode));""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,AVG(numOrder)
0,326.0


## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [7]:
#Your code here; use a subquery. No join will be necessary.
c.execute('''SELECT customerNumber, contactLastName, contactFirstName
                    FROM customers
                    WHERE customerNumber in (select customerNumber FROM orders
                    WHERE orderDate = '2003-01-31')
            ''')

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [8]:
#Your code here
c.execute('''SELECT productName,
                    count(distinct orderNumber) as numberOrders,
                    sum(quantityOrdered) as totalUnitsSold
                    from products
                    join orderdetails using (productCode)
                    group by productName
                    order by totalUnitsSold DESC''')

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productName,numberOrders,totalUnitsSold
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [9]:
#Your code here
c.execute('''SELECT  productName,
                     count(distinct customerNumber) numPurchasers
                     from products 
                     join orderdetails using (productCode)
                     join orders using (orderNumber)
                     group by productName
                     order by numPurchasers DESC
                     ''')


df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productName,numPurchasers
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [11]:
#Your code here


df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

TypeError: 'NoneType' object is not iterable

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [20]:
#Your code here

c.execute("""select employeeNumber,
                    firstName,
                    lastName,
                    count(distinct customerNumber)
                    from employees e
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    group by 1,2,3
                    having AVG(creditLimit) > 15000;
          
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,employeeNumber,firstName,lastName,count(distinct customerNumber)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!